

```

```

## **TASK 1: EDA + Preprocessing**

# CODE

In [ ]:
import pandas as pd

df = pd.read_csv("/appl_accepted_20072019Q3.csv", low_memory=False)


In [ ]:
# Filter only needed statuses
df = df[df['loan_status'].isin(['Fully Paid','Charged Off'])]

# Binary Target Mapping
df['target'] = df['loan_status'].map({'Fully Paid':0, 'Charged Off':1})

# Select Features
features = ['loan_amnt','int_rate','annual_inc','dti','emp_length',
            'term','home_ownership','purpose','grade','sub_grade',
            'verification_status','delinq_2yrs','revol_util','total_acc']
df = df[features + ['target']]

# Handle Missing & Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['home_ownership','purpose','grade','sub_grade','verification_status','term', 'emp_length']
for c in cat_cols:
    df[c] = LabelEncoder().fit_transform(df[c].astype(str))

df.fillna(df.median(), inplace=True)


# Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

df.to_csv("processed_loan_data.csv", index=False)


# **TASK 2: Deep Learning Classification**

# Code

In [ ]:
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score, f1_score

class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

model = MLP(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)



# Train Loop

In [ ]:
for epoch in range(20):
    model.train()
    optimizer.zero_grad()
    pred = model(torch.tensor(X_train.values, dtype=torch.float32)).squeeze()
    loss = criterion(pred, torch.tensor(y_train.values, dtype=torch.float32))
    loss.backward()
    optimizer.step()
    print(epoch, loss.item())


0 59.446720123291016
1 38.8918342590332
2 25.732641220092773
3 21.297805786132812
4 20.322559356689453
5 20.08755874633789
6 20.019641876220703
7 19.99201202392578
8 19.99458885192871
9 19.987287521362305
10 19.98943519592285
11 19.986858367919922
12 19.98814582824707
13 19.98814582824707
14 19.98814582824707
15 19.98814582824707
16 19.98814582824707
17 19.98814582824707
18 19.98814582824707
19 19.98814582824707


# Evaluation

In [ ]:
model.eval()
pred_test = model(torch.tensor(X_test.values, dtype=torch.float32)).detach().numpy()
auc = roc_auc_score(y_test, pred_test)
f1 = f1_score(y_test, pred_test.round())
print("Test AUC:", auc)
print("Test F1:", f1)


Test AUC: 0.5
Test F1: 0.0


# **TASK 3: Offline RL Agent**

# Define RL Dataset


In [ ]:
import numpy as np

df_rl = df.copy()
df_rl['reward'] = df_rl.apply(lambda r:
                              0 if r['target']==1 else
                              r['loan_amnt'] * r['int_rate'] if r['target']==0 else
                              -r['loan_amnt'], axis=1)

states = df_rl[features].values
actions = np.where(df_rl['target']==0, 1, 1) # historical always approved
rewards = df_rl['reward'].values

In [ ]:
pip install d3rlpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827728 sha256=68fa8e4e9e7d85c10850b7015cedc2c8d49da30a0be7cd8288416a06d3259418
  Stored in directory: /root/.cache/pip/wheels/95/51/6c/9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.2.2
    Uninstalling gymnasium-1.2.2:
      Successfully uninstalled gymnasium-1.2.2
  Attempting uninstall: gy

In [ ]:
from d3rlpy.algos import DiscreteCQL, DiscreteCQLConfig
from d3rlpy.dataset import MDPDataset
import torch

dataset = MDPDataset(states, actions, rewards, terminals=np.ones(len(states)))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

cql_config = DiscreteCQLConfig() # Create a default config object for DiscreteCQL
cql = DiscreteCQL(config=cql_config, device=device, enable_ddp=False)
cql.fit(dataset, n_steps=20000)

2025-12-09 20:35.24 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(14,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-12-09 20:35.24 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-12-09 20:35.26 [info     ] Action size has been automatically determined. action_size=2
2025-12-09 20:35.26 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(14,)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=2)
2025-12-09 20:35.26 [debug    ] Building models...            
2025-12-09 20:35.26 [debug    ] Models have been built.       
2025-12-09 20:35.26 [info  

Epoch 1/2:   0%|          | 0/10000 [00:00<?, ?it/s]

2025-12-09 20:36.53 [info     ] DiscreteCQL_20251209203526: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.0011439874887466432, 'time_algorithm_update': 0.007227797961235047, 'loss': 72264.63454589844, 'td_loss': 72264.63454589844, 'conservative_loss': 0.0, 'time_step': 0.008606693887710572} step=10000
2025-12-09 20:36.53 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251209203526/model_10000.d3


Epoch 2/2:   0%|          | 0/10000 [00:00<?, ?it/s]

2025-12-09 20:38.27 [info     ] DiscreteCQL_20251209203526: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.0012074750185012818, 'time_algorithm_update': 0.0077506826877594, 'loss': 68763.53992460937, 'td_loss': 68763.41676210938, 'conservative_loss': 0.1231626708984375, 'time_step': 0.009257170701026916} step=20000
2025-12-09 20:38.27 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251209203526/model_20000.d3


[(1,
  {'time_sample_batch': 0.0011439874887466432,
   'time_algorithm_update': 0.007227797961235047,
   'loss': 72264.63454589844,
   'td_loss': 72264.63454589844,
   'conservative_loss': 0.0,
   'time_step': 0.008606693887710572}),
 (2,
  {'time_sample_batch': 0.0012074750185012818,
   'time_algorithm_update': 0.0077506826877594,
   'loss': 68763.53992460937,
   'td_loss': 68763.41676210938,
   'conservative_loss': 0.1231626708984375,
   'time_step': 0.009257170701026916})]

# Evaluate Policy


In [ ]:
import numpy as np
from d3rlpy.dataset import MDPDataset

# Filter df_rl to get the rows corresponding to X_test
df_test_rl = df_rl.loc[X_test.index]

# Prepare test states, actions, and rewards
states_test = df_test_rl[features].values
actions_test = np.where(df_test_rl['target']==0, 1, 1) # historical always approved
rewards_test = df_test_rl['reward'].values

# Create an MDPDataset for the test set
test_dataset = MDPDataset(states_test, actions_test, rewards_test, terminals=np.ones(len(states_test)))

# Evaluate the policy on the test dataset
predicted_actions_test = cql.predict(states_test)
q_values_test = cql.predict_value(states_test, predicted_actions_test)
estimated_policy_value_test = np.mean(q_values_test)
print("Estimated Policy Value on Test Set:", estimated_policy_value_test)

2025-12-09 20:39.58 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(14,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-12-09 20:39.58 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-12-09 20:39.58 [info     ] Action size has been automatically determined. action_size=2
Estimated Policy Value on Test Set: 140121.66
